### Web Map Deliverable
In this notebook, I'll be preparing the data - AMPA Data Sample - for use in ArcGIS. The final web app can be found [here](https://lstrachan.maps.arcgis.com/apps/instant/slider/index.html?appid=5b02eae0a86f438783c6b082ab4a5209&locale=en-us).

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('AMPA_Data_Sample.xlsx - invasive_species.csv')

In [4]:
data.head()

,Datetime,Tide,Weather Condition,Water temperature (ºC),Zone,Supratidal/Middle Intertidal,Substrate,Asparagopsis armata (tufosa),Asparagopsis armata (adulta),Osmundea pinnatifida,Cladophora sp. (limo),Codium sp. (alga verde carnuda),Colpomenia sinuosa (alga bolhas),Sessile Coverage,Total Mobile Species,Abundance (ind/m2)
0,11/28/2011 10:10:00,0.6,Clear sky,16,D,Medium,Puddle,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,6645.00%,4.0,0.05
1,11/28/2011 10:25:00,0.6,Clear sky,16,D,Medium,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,7510.00%,2.0,0.10
2,11/28/2011 10:40:00,0.6,Clear sky,16,D,Medium,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,6720.00%,1.0,0.00
3,11/28/2011 11:00:00,0.6,Clear sky,16,E,Medium,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,2.0,0.20
4,11/28/2011 11:15:00,0.6,Clear sky,16,E,Medium,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,4825.00%,0.0,0.00


In [5]:
# Creating an 'Id' column to be able to join in ArcGIS
data['Id'] = 0

In [6]:
data.head(1)

,Datetime,Tide,Weather Condition,Water temperature (ºC),Zone,Supratidal/Middle Intertidal,Substrate,Asparagopsis armata (tufosa),Asparagopsis armata (adulta),Osmundea pinnatifida,Cladophora sp. (limo),Codium sp. (alga verde carnuda),Colpomenia sinuosa (alga bolhas),Sessile Coverage,Total Mobile Species,Abundance (ind/m2),Id
0,11/28/2011 10:10:00,0.6,Clear sky,16,D,Medium,Puddle,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,6645.00%,4.0,0.05,0


In [7]:
data['Supratidal/Middle Intertidal'].unique()

array(['Medium', 'Supra', 'Super'], dtype=object)

In [8]:
# checking how many "Super"s there are:
data[data['Supratidal/Middle Intertidal'] ==  'Super']

,Datetime,Tide,Weather Condition,Water temperature (ºC),Zone,Supratidal/Middle Intertidal,Substrate,Asparagopsis armata (tufosa),Asparagopsis armata (adulta),Osmundea pinnatifida,Cladophora sp. (limo),Codium sp. (alga verde carnuda),Colpomenia sinuosa (alga bolhas),Sessile Coverage,Total Mobile Species,Abundance (ind/m2),Id
1808,1/25/2019 9:00:00,0.9,Sunny,15,B,Super,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,3420.00%,0.0,0.0,0


In [9]:
# Here I am replacing the one rogue "Supra" with "Super" instead
data.replace('Super', 'Supra', inplace=True)

In [10]:
# Checking to confirmed that worked
data['Supratidal/Middle Intertidal'].unique()

array(['Medium', 'Supra'], dtype=object)

In [11]:
data['Supratidal/Middle Intertidal'].value_counts()

Medium    1242
Supra      707
Name: Supratidal/Middle Intertidal, dtype: int64

In [12]:
# Creating a function to make individual datasets for each zone
# this will enable me to join this info to the ArcGIS shapefiles for the corresponding zones

def zones_datasets(df):
    for zone in df['Zone'].unique():
        for tide in df['Supratidal/Middle Intertidal'].unique():
            name = f'{zone}{tide}.csv'
            isolated_df = data.loc[(data['Zone'] == zone) & (data['Supratidal/Middle Intertidal'] == tide)]
            isolated_df.to_csv(name, index= False)

In [13]:
# Running the function on our dataframe
zones_datasets(data)

In [14]:
# Confirming that the function worked
AMedium = pd.read_csv('AMedium.csv')
AMedium.head()

,Datetime,Tide,Weather Condition,Water temperature (ºC),Zone,Supratidal/Middle Intertidal,Substrate,Asparagopsis armata (tufosa),Asparagopsis armata (adulta),Osmundea pinnatifida,Cladophora sp. (limo),Codium sp. (alga verde carnuda),Colpomenia sinuosa (alga bolhas),Sessile Coverage,Total Mobile Species,Abundance (ind/m2),Id
0,11/29/2011 11:10:00,0.8,Clear sky,16,A,Medium,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,1980.00%,7.0,0.0,0
1,11/29/2011 11:25:00,0.8,Clear sky,16,A,Medium,Puddle/Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,4270.00%,15.0,0.4,0
2,11/29/2011 11:40:00,0.8,Clear sky,16,A,Medium,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,9700.00%,4.0,0.0,0
3,11/29/2011 12:00:00,0.8,Clear sky,16,A,Medium,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,1500.00%,2.0,0.0,0
4,12/14/2011 11:45:00,0.9,Clear sky,16,A,Medium,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,20.00%,0.0,0.0,0


In [16]:
# The E Supra shapefile ID is 7 instead of 0, so we need to amend this csv:
ESupra = pd.read_csv('ESupra.csv')
ESupra.head()

,Datetime,Tide,Weather Condition,Water temperature (ºC),Zone,Supratidal/Middle Intertidal,Substrate,Asparagopsis armata (tufosa),Asparagopsis armata (adulta),Osmundea pinnatifida,Cladophora sp. (limo),Codium sp. (alga verde carnuda),Colpomenia sinuosa (alga bolhas),Sessile Coverage,Total Mobile Species,Abundance (ind/m2),Id
0,11/28/2011 12:00:00,0.6,Clear sky,16,E,Supra,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,9700.00%,4.0,0.0,0
1,11/28/2011 12:15:00,0.6,Clear sky,16,E,Supra,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,5110.00%,0.0,0.0,0
2,12/12/2011 12:00:00,0.9,Clear sky,16,E,Supra,Rock,0.00%,0.00%,0.00%,9500.00%,0.00%,0.00%,7185.00%,0.0,0.0,0
3,12/12/2011 12:15:00,0.9,Clear sky,16,E,Supra,Rock,0.00%,0.00%,0.00%,1500.00%,0.00%,0.00%,7655.00%,0.0,0.2,0
4,1/11/2012 11:30:00,0.7,Clear sky,15,E,Supra,Sand,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,5370.00%,0.0,0.0,0


In [17]:
ESupra['Id'] = 7

In [19]:
# Confirming this worked, and yes it did:
ESupra.head(2)

,Datetime,Tide,Weather Condition,Water temperature (ºC),Zone,Supratidal/Middle Intertidal,Substrate,Asparagopsis armata (tufosa),Asparagopsis armata (adulta),Osmundea pinnatifida,Cladophora sp. (limo),Codium sp. (alga verde carnuda),Colpomenia sinuosa (alga bolhas),Sessile Coverage,Total Mobile Species,Abundance (ind/m2),Id
0,11/28/2011 12:00:00,0.6,Clear sky,16,E,Supra,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,9700.00%,4.0,0.0,7
1,11/28/2011 12:15:00,0.6,Clear sky,16,E,Supra,Rock,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,5110.00%,0.0,0.0,7


In [21]:
# Re-saving the CSV:
ESupra.to_csv('ESupra_fixed.csv', index=False)